## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/2-create-nn.html)


### Step2

In [19]:
import mxnet as mx
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

mx.__version__

'1.8.0'

## Step 2 - Create neural network
Uma camada com 3 entradas e 5 saidas, com função de ativação "RELU"


In [20]:
layer=nn.Dense(5,in_units=3,activation='relu')
layer

Dense(3 -> 5, Activation(relu))

In [21]:
layer.initialize()

#### dez linhas com três parametros

In [22]:
x=np.random.uniform(-1,1,(10,3))
x

array([[-0.28098422,  0.22612691, -0.12593609],
       [ 0.80469716,  0.39526236, -0.8014393 ],
       [-0.8795491 ,  0.9396181 ,  0.3335334 ],
       [ 0.30628002,  0.3412757 , -0.65818083],
       [-0.57923484, -0.28369564, -0.74214745],
       [ 0.50137234, -0.3691433 ,  0.21566129],
       [-0.27257848, -0.34990555,  0.1403935 ],
       [-0.92314917, -0.12279695,  0.26854813],
       [ 0.97674763,  0.91789854, -0.79591036],
       [ 0.30558062, -0.5822465 ,  0.27011776]])

In [23]:
layer(x)

array([[0.01974824, 0.01128841, 0.02222548, 0.        , 0.        ],
       [0.01278322, 0.        , 0.        , 0.        , 0.06103281],
       [0.02687198, 0.04161403, 0.04491762, 0.        , 0.        ],
       [0.02472852, 0.        , 0.0159209 , 0.        , 0.0305621 ],
       [0.05816879, 0.03378498, 0.05280771, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.027472  , 0.01735162],
       [0.00029454, 0.02929036, 0.00241975, 0.00982013, 0.        ],
       [0.02174219, 0.06963804, 0.03290942, 0.00352226, 0.        ],
       [0.01068064, 0.        , 0.        , 0.        , 0.0839951 ],
       [0.        , 0.        , 0.        , 0.03103637, 0.00050997]])

In [24]:
layer.params

dense7_ (
  Parameter dense7_weight (shape=(5, 3), dtype=float32)
  Parameter dense7_bias (shape=(5,), dtype=float32)
)

#### pesos e bias de entrada de cada elemento da camada interna de 5 neuronios.

In [25]:
layer.weight.data()

array([[-0.03968295,  0.00958075, -0.05106945],
       [-0.06736943, -0.02462026,  0.01646897],
       [-0.04904552,  0.0156934 , -0.03887501],
       [ 0.01637076, -0.01589154,  0.06212472],
       [ 0.05636378,  0.02545484, -0.007007  ]])

In [26]:
layer.bias.data()


array([0., 0., 0., 0., 0.])

Chain layers into a neural network using nn.Sequential

Sequential é a ligação de camadas onde a saída de uma camada é a entrada de outra

In [27]:
net = nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),nn.Dense(25,activation='relu'),nn.Dense(2))

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [28]:
net[1]

Dense(-1 -> 25, Activation(relu))

#### Custom neural network architecture flexibly

In [29]:
class Net(nn.Block):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x

In [30]:
class MLP(nn.Block):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(5,activation='relu')
        self.dense2=nn.Dense(25,activation='relu')
        self.dense3=nn.Dense(2)
    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

In [31]:
net=MLP()
net

MLP(
  (dense1): Dense(-1 -> 5, Activation(relu))
  (dense2): Dense(-1 -> 25, Activation(relu))
  (dense3): Dense(-1 -> 2, linear)
)

In [32]:
net.dense1.params

dense11_ (
  Parameter dense11_weight (shape=(5, -1), dtype=float32)
  Parameter dense11_bias (shape=(5,), dtype=float32)
)

#### Creating custom layers using Parameters

In [33]:
from mxnet.gluon import Parameter

weight=Parameter("custom_parameter_weight",shape=(5,-1))
bias=Parameter("custom_parameter_bias",shape=(5,-1))

weight,bias

(Parameter custom_parameter_weight (shape=(5, -1), dtype=<class 'numpy.float32'>),
 Parameter custom_parameter_bias (shape=(5, -1), dtype=<class 'numpy.float32'>))

Custom layer com função linear sem função de ativação. 

w*x+b

In [34]:
class custom_layer(nn.Block):
    def __init__(self,out_units, in_units=0):
        super().__init__()
        self.weight=Parameter("weight",shape=(in_units,out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
    def forward(self,x):
        return np.dot(x,self.weight.data())+self.bias.data()

In [35]:
dense=custom_layer(3,in_units=5)
dense.initialize()
dense(np.random.uniform(size=(4,5)))

RuntimeError: Parameter 'weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

### [LeNet](http://yann.lecun.com/exdb/lenet/)

In [ ]:
class LeNet(nn.Block):
    def __init__(self):
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        